# DCF with ESG premium - Calculate the fair value of FAANGS and add premium for ESG scores

## Part 1
Import data from the CSV files with FAANG data and convert to pandas DataFrames.

In [62]:
#Import necessary modules
import pandas as pd
from pathlib import Path
%matplotlib inline

In [63]:
#Convert the FAANG data csv to a datframe.
faang_data_df = pd.read_csv(
    Path('./Resources/FAANG_DATA/stock_data.csv'),
    index_col="TICKER", 
)
faang_data_df.head()

,DEBT,CASH & EQUIVALENTS,MARGIN OF SAFETY,ESG SCORE
TICKER,,,,
META,16680,12681,45,1007.00
AAPL,119690,27502,45,886.00
AMZN,157560,37478,45,1428.00
NFLX,16910,5819,45,NaN
GOOG,28810,17936,45,1034.00


In [64]:
#Drop incomplete data
faang_data_df = faang_data_df.drop(index='NFLX')
faang_data_df

,DEBT,CASH & EQUIVALENTS,MARGIN OF SAFETY,ESG SCORE
TICKER,,,,
META,16680,12681,45,1007.00
AAPL,119690,27502,45,886.00
AMZN,157560,37478,45,1428.00
GOOG,28810,17936,45,1034.00


In [65]:
#Normalize the ESG scores with Z-Scores

esg_mean = faang_data_df['ESG SCORE'].mean()
esg_std = faang_data_df['ESG SCORE'].std()
esg_new =  (faang_data_df.loc[:,'ESG SCORE']  - esg_mean)/esg_std
faang_data_df['ESG SCORE'] = esg_new*0.1
faang_data_df

,DEBT,CASH & EQUIVALENTS,MARGIN OF SAFETY,ESG SCORE
TICKER,,,,
META,16680,12681,45,-0.03
AAPL,119690,27502,45,-0.09
AMZN,157560,37478,45,0.14
GOOG,28810,17936,45,-0.02


In [66]:
#Convert the individual FAANG company csv files to datframes.
meta_df = pd.read_csv(
    Path('./FAANG_DATA/meta.csv'),
    index_col="YEAR", 
)

aapl_df = pd.read_csv(
    Path('./FAANG_DATA/aapl.csv'),
    index_col="YEAR", 
)

amzn_df = pd.read_csv(
    Path('./FAANG_DATA/amzn.csv'),
    index_col="YEAR", 
)


goog_df = pd.read_csv(
    Path('./FAANG_DATA/goog.csv'),
    index_col="YEAR", 
)


In [67]:
#preview one of the dataframes.
amzn_df.head()

,2017,2018,2019,2020,2021
YEAR,,,,,
PERIOD,1.00,2.00,3.00,4.00,5.00
FCF,6410.00,17296.00,21653.00,25924.00,-14726.00
WACC,0.10,0.13,0.11,0.08,0.07


## Calculate the DCFs for each company

### META

In [68]:
#Discount the cash flows for META from 2017-2020.
meta_growth_pv = meta_df.loc["FCF"]/((1+meta_df.loc["WACC"])**meta_df.loc["PERIOD"])
meta_growth_pv = meta_growth_pv.drop('2021')

In [69]:
#calculate the Terminal Value and discount to present value. Add the discounted cashflows and the discounted terminal value.
meta_perpetuity_g = 0.05
meta_perpetuity_pv = (meta_df.at['FCF', '2021']*(1+meta_perpetuity_g))/(meta_df.at['WACC', '2021']- meta_perpetuity_g)
meta_wacc = meta_df.at['WACC', '2021']
dcf_meta_perpetuity_pv = meta_perpetuity_pv/(1+(meta_wacc))**5
dcf_meta_perpetuity_pv

558702.5089439774

In [70]:
#Preview the sum of all discounted cash flows
meta_dcf = meta_growth_pv.sum() + dcf_meta_perpetuity_pv
print(meta_dcf)
#Add esg premium, which will be a percent increase of the normalized ESG score. Convert MOS to percentage and divide by two (upside and downside margin).
meta_esg = faang_data_df.loc["META", "ESG SCORE"]
print(meta_esg)
meta_mos = (faang_data_df.loc["META", "MARGIN OF SAFETY"] * 0.01)/2
print(meta_mos)

621502.3783077226
-0.03476586277949939
0.225


In [71]:
#Subtract debt, add cash & equivalents. Apply margin. of safety for a range. 
meta_dcf = meta_dcf - faang_data_df.loc["META", "DEBT"] + faang_data_df.loc["META", "CASH & EQUIVALENTS"]
meta_dcf = meta_dcf*(1+meta_esg)
print(meta_dcf)
meta_dcf_low = meta_dcf*(1-meta_mos)
print(meta_dcf_low)
meta_dcf_high = meta_dcf*(1+meta_mos)
print(meta_dcf_high)

596035.3405915991
461927.38895848935
730143.292224709


### AAPL

In [72]:
#Discount the cash flows for AAPL from 2017-2020.
aapl_growth_pv = aapl_df.loc["FCF"]/((1+aapl_df.loc["WACC"])**aapl_df.loc["PERIOD"])
aapl_growth_pv = aapl_growth_pv.drop('2021')

In [73]:
#calculate the Terminal Value and discount to present value. Add the discounted cashflows and the discounted terminal value.
aapl_perpetuity_g = 0.05
aapl_perpetuity_pv = (aapl_df.at['FCF', '2020']*(1+aapl_perpetuity_g))/(aapl_df.at['WACC', '2021']- aapl_perpetuity_g)
aapl_wacc = aapl_df.at['WACC', '2021']
dcf_aapl_perpetuity_pv = aapl_perpetuity_pv/(1+(aapl_wacc))**5

In [74]:
#Preview the sum of all discounted cash flows
aapl_dcf = aapl_growth_pv.sum() + dcf_aapl_perpetuity_pv
aapl_dcf

1510660.4295669312

In [75]:
#Add esg premium, which will be a percent increase of the normalized ESG score. Convert MOS to percentage.
aapl_esg = faang_data_df.loc["AAPL", "ESG SCORE"]

aapl_mos = (faang_data_df.loc["AAPL", "MARGIN OF SAFETY"] * 0.01)/2

In [76]:
#Subtract debt, add cash & equivalents. Apply margin. of safety for a range. 
aapl_dcf = aapl_dcf - faang_data_df.loc["AAPL", "DEBT"] + faang_data_df.loc["AAPL", "CASH & EQUIVALENTS"]
aapl_dcf = aapl_dcf*(1+aapl_esg)
aapl_dcf_low = aapl_dcf*(1-aapl_mos)
aapl_dcf_high = aapl_dcf*(1+aapl_mos)
aapl_dcf

1296166.6409720299

### AMZN

In [77]:
#Discount the cash flows for AAPL from 2017-2020.
amzn_growth_pv = amzn_df.loc["FCF"]/((1+amzn_df.loc["WACC"])**amzn_df.loc["PERIOD"])
amzn_growth_pv = amzn_growth_pv.drop('2021')
amzn_growth_pv

2017    5812.48
2018   13452.29
2019   15700.57
2020   19275.27
dtype: float64

In [78]:
amzn_positive_fcf = (0.2*amzn_df.at['FCF', '2021'] + 0.8*amzn_df.at['FCF', '2020'])
amzn_positive_fcf

17794.0

In [79]:
#Calculate the Terminal Value and discount to present value. Add the discounted cashflows and the discounted terminal value.
amzn_perpetuity_g = 0.05
amzn_perpetuity_pv = (amzn_positive_fcf*(1+amzn_perpetuity_g))/(amzn_df.at['WACC', '2021']- amzn_perpetuity_g)
amzn_wacc = amzn_df.at['WACC', '2021']
dcf_amzn_perpetuity_pv = amzn_perpetuity_pv/(1+(amzn_wacc))**5
print(amzn_perpetuity_pv)

1139250.0000000002


In [80]:
#Preview the sum of all discounted cash flows
amzn_dcf = amzn_growth_pv.sum() + dcf_amzn_perpetuity_pv
amzn_dcf

880313.4734454678

In [81]:
#Add esg premium, which will be a percent increase of the normalized ESG score. Convert MOS to percentage.
amzn_esg = faang_data_df.loc["AMZN", "ESG SCORE"]

amzn_mos = (faang_data_df.loc["AMZN", "MARGIN OF SAFETY"] * 0.01)/2

In [82]:
#Subtract debt, add cash & equivalents. Apply margin. of safety for a range. 
amzn_dcf = amzn_dcf - faang_data_df.loc["AMZN", "DEBT"] + faang_data_df.loc["AMZN", "CASH & EQUIVALENTS"]
amzn_dcf = amzn_dcf*(1+amzn_esg)
amzn_dcf_low = amzn_dcf*(1-amzn_mos)
amzn_dcf_high = amzn_dcf*(1+amzn_mos)

amzn_dcf

869912.359954118

### GOOG

In [83]:
#Discount the cash flows for META from 2017-2020.
goog_growth_pv = goog_df.loc["FCF"]/((1+goog_df.loc["WACC"])**goog_df.loc["PERIOD"])
goog_growth_pv = goog_growth_pv.drop('2021')

In [84]:
#calculate the Terminal Value and discount to present value. Add the discounted cashflows and the discounted terminal value.
goog_perpetuity_g = 0.05
goog_perpetuity_pv = (goog_df.at['FCF', '2021']*(1+goog_perpetuity_g))/(goog_df.at['WACC', '2021']- goog_perpetuity_g)
goog_wacc = goog_df.at['WACC', '2021']
dcf_goog_perpetuity_pv = goog_perpetuity_pv/(1+(goog_wacc))**5
dcf_goog_perpetuity_pv

2080552.922620001

In [85]:
#Preview the sum of all discounted cash flows
goog_dcf = goog_growth_pv.sum() + dcf_goog_perpetuity_pv
print(goog_dcf)
#Add esg premium, which will be a percent increase of the normalized ESG score. Convert MOS to percentage.
goog_esg = faang_data_df.loc["GOOG", "ESG SCORE"]
print(goog_esg)
goog_mos = (faang_data_df.loc["GOOG", "MARGIN OF SAFETY"] * 0.01)/2
print(goog_mos)

2179049.5819982053
-0.023283559476178495
0.225


In [86]:
#Subtract debt, add cash & equivalents. Apply margin. of safety for a range. 
goog_dcf = goog_dcf - faang_data_df.loc["GOOG", "DEBT"] + faang_data_df.loc["GOOG", "CASH & EQUIVALENTS"]
goog_dcf = goog_dcf*(1+goog_esg)
print(goog_dcf)
goog_dcf_low = goog_dcf*(1-goog_mos)
print(goog_dcf_low)
goog_dcf_high =goog_dcf*(1+goog_mos)
print(goog_dcf_high)

2117692.7368799523
1641211.871081963
2594173.602677942


## Print the results

### META

In [87]:
print(f"The fair value of META with an ESG premium of {meta_esg}% and a 45% margin of satefy should fall between:")
print(f"${meta_dcf_low:.2f} and ${meta_dcf_high:.2f}.")
print(f"Our mean fair value is ${meta_dcf:.2f}.")

The fair value of META with an ESG premium of -0.03476586277949939% and a 45% margin of satefy should fall between:
$461927.39 and $730143.29.
Our mean fair value is $596035.34.


### AAPL

In [88]:
print(f"The fair value of AAPL with an ESG premium of {aapl_esg}% and a 45% margin of satefy should fall between:")
print(f"${aapl_dcf_low:.2f} and ${aapl_dcf_high:.2f}.")
print(f"Our mean fair value is ${aapl_dcf:.2f}.")

The fair value of AAPL with an ESG premium of -0.08622359239808565% and a 45% margin of satefy should fall between:
$1004529.15 and $1587804.14.
Our mean fair value is $1296166.64.


### AMZN

In [89]:
print(f"The fair value of AMZN with an ESG premium of {amzn_esg}% and a 45% margin of satefy should fall between:")
print(f"${amzn_dcf_low:.2f} and ${amzn_dcf_high:.2f}.")
print(f"Our mean fair value is ${amzn_dcf:.2f}.")

The fair value of AMZN with an ESG premium of 0.14427301465376352% and a 45% margin of satefy should fall between:
$674182.08 and $1065642.64.
Our mean fair value is $869912.36.


### GOOG

In [90]:
print(f"The fair value of GOOG with an ESG premium of {goog_esg}% and a 45% margin of satefy should fall between:")
print(f"${goog_dcf_low:.2f} and ${goog_dcf_high:.2f}.")
print(f"Our mean fair value is ${goog_dcf:.2f}.")

The fair value of GOOG with an ESG premium of -0.023283559476178495% and a 45% margin of satefy should fall between:
$1641211.87 and $2594173.60.
Our mean fair value is $2117692.74.


## Save the results

In [91]:
pd.set_option('display.float_format', '{:.2f}'.format)
data = [[meta_esg, aapl_esg, amzn_esg, goog_esg],
        [45, 45, 45, 45],
        [meta_dcf_low, aapl_dcf_low, amzn_dcf_low, goog_dcf_low],
        [meta_dcf_high, aapl_dcf_high, amzn_dcf_high, goog_dcf_high],
        [meta_dcf, aapl_dcf, amzn_dcf, goog_dcf]]
res_df = pd.DataFrame(
    data,
    index = ['esg_prem', 'margin_of_safety', 'dcf_low', 'dcf_high', 'dcf'],
    columns = [ 'META', 'AAPL', 'AMZN', 'GOOG']
)
res_df.to_csv('./Resources/FAANG_DATA/res.csv')